In [ ]:
# !pip install -q rpy2==3.5.1
!pip install -q --upgrade rpy2

In [ ]:
%load_ext rpy2.ipython

In [ ]:
'''
Authors: Daniel M. Low
License: See license in github repository
'''

import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import recall_score
import os



project_name = 'vfp'
filenames = ['annotations_ps_22-05-29T23-42-51.csv',
             'annotations_cjs_22-10-19T17-48-21.csv', 
             'annotations_ch_22-10-16T01-03-17.csv']
# pd.options.display.width = 0
pd.set_option("display.max_columns", None)
on_colab = False
if on_colab:
  from google.colab import drive
  drive.mount('/content/drive')
  input_dir = f'/content/drive/MyDrive/datum/{project_name}/data/output/annotations/'
  output_dir = f'/content/drive/MyDrive/datum/{project_name}/data/output/annotations/'
else:
  input_dir = './data/input/annotations/'
  output_dir = './data/output/annotations/'




# Observe responses, fix formatting, identify mislabelled samples

In [ ]:
variables = pd.read_csv(input_dir + 'annotations_ch_22-10-16T01-03-17.csv', index_col = 0).columns.tolist()
variables.remove('file')
variables

In [ ]:
for variable in variables:
  print(f'\n\n===={variable}====')
  
  counts_all = []
  for filename in filenames:
    annotator = filename.split('_')[1]
    df = pd.read_csv(input_dir + filename, index_col = 0)
    values = np.sort(df[variable].unique().tolist())
    print(annotator, ':', values)
    counts_i = pd.DataFrame(df[variable].value_counts())
    
    counts_i.columns = [variable+'_'+annotator]
    counts_i.index = [str(n) for n in counts_i.index]
    counts_all.append(counts_i)
    print()

  counts_all2 = pd.concat(counts_all,axis=1,ignore_index=False)
  display(counts_all2)

PS asked made the following typos


134: VFP = 1, NaN for all other values (or he would need to redo)


77: loudness (recording)


In [ ]:
print(df.loc[134].values)
print([df.loc[134].values[0]] + ([np.nan]*(len(df.loc[134].values)-1)))
df.loc[134] = [df.loc[134].values[0]] + ([np.nan]*(len(df.loc[134].values)-1))
df.loc[134, 'vfp'] = 1
df.loc[134, 'noise'] = 2 #same as before
print(df.loc[134].values)

In [ ]:
dfs = {}


for filename in filenames:
  annotator = filename.split('_')[1]
  print('===', filename)
  df = pd.read_csv(input_dir + filename, index_col = 0)
  df['noise'] = df['noise'].replace('o', 0).replace('99', np.nan).replace('99.0', np.nan).replace(99, np.nan)
  df['vfp'] = df['vfp'].replace(2, 1)
  df['loudness (recording)'] = df['loudness (recording)'].replace(20, np.nan).replace(10, np.nan).replace(0, np.nan)
  df['strain'] = df['strain'].replace('220', '20').replace("22\\5", '25')
  variables = df.columns
  for variable in variables:
    try: df[variable] =df[variable].astype(float) 
    except: 
      df[variable] =df[variable].astype(object) #for string columns

  if annotator == 'ps':
    # typos asked to be corrected
    assert df.loc[134, 'vfp'] == 0
    df.loc[134] = [df.loc[134].values[0]] + ([np.nan]*(len(df.loc[134].values)-1)) # setting NaN to other values which would need to be corrected
    df.loc[134, 'vfp'] = 1
    df.loc[134, 'noise'] = 2 #same as before
    print(df.loc[134].values)
    
    print("df.loc[77, 'loudness (recording)']", df.loc[77, 'loudness (recording)'])
    df.loc[77, 'loudness (recording)'] = 3


  
  y_true = [0 if 'Norm' in n else 1 for n in df.file.values ]
  df['y_true'] = y_true
  
  dfs[annotator] = df.sort_values('file').reset_index(drop = True)


  


# Plot performance split by annotator

In [ ]:

  plt.rcParams["figure.figsize"] = (12,8)
  plt.rcParams.update({'font.size': 18})

In [ ]:
run_this = False

if run_this:
  plt.rcParams["figure.figsize"] = (12,8)
  plt.rcParams.update({'font.size': 18})


  for variable in df.columns:
    if variable in ['file', 'y_true']:
        continue
    for i, (annotator, df) in enumerate(dfs.items()):
      if annotator == 'ps':
        annotator = 'Annotator 1'
      elif annotator == 'cjs':
        annotator = 'Annotator 2'
      elif annotator == 'ch':
        annotator = 'Annotator 3'
      
      
      

      
      if variable in ['noise', 'vfp', 'loudness (recording)']:
        plt.subplot(1, 3, i+1)
        # df.groupby('y_true')[variable].value_counts().unstack(0).plot.bar()
        # plt.title(f'{annotator}', size = 20)  
        # plt.xticks(rotation = 0)
        # plt.legend()
        bar_plot_df = df.groupby('y_true')[variable].value_counts().unstack(0)
        x_ticks = bar_plot_df.index.values
        control_values = bar_plot_df[0].values
        vfp_values = bar_plot_df[1].values
        
        width = 0.35
        rects1 = plt.bar(x_ticks - width/2, control_values, width, label='Controls')
        rects2 = plt.bar(x_ticks + width/2, vfp_values, width, label='VFP')
        plt.title(f'{annotator}', size = 20)  
        if i+1==1:
          plt.ylabel('Count')
        if i+1==2:
          plt.xlabel(variable.capitalize())
        if i+1==3:
          plt.legend()
        plt.ylim(0,76)
          
        

      else:
        # variables that range from 0 to 100
        plt.subplot(1, 3, i+1)
        x0 = df[df['y_true']==0][variable].values
        x1 = df[df['y_true']==1][variable].values
        alpha = 0.7
        plt.hist(x0, bins=10, alpha=alpha, label = 'Controls')
        plt.hist(x1, bins=10, alpha=alpha, label = 'VFP')
        # plt.legend()
        plt.title(f'{annotator}', size = 20)  
        if i+1==1:
          plt.ylabel('Count')
        if i+1==2:
          plt.xlabel(variable.capitalize())
        if i+1==3:
          plt.legend()
        plt.ylim(0,76)

          
    plt.show()
    
    




In [ ]:
min_and_max = True
plt.rcParams["figure.figsize"] = (6,12)
plt.rcParams.update({'font.size': 16})
plt.style.use('default')

for i, variable in enumerate(['vfp', 'noise','loudness (recording)']):
  plt.subplot(1, 3, i+1)
  if variable in ['file', 'y_true']:
        continue



  control_values_all = []
  vfp_values_all = []
  for j, (annotator, df) in enumerate(dfs.items()):
   
    

    if variable in ['noise', 'vfp', 'loudness (recording)']:
      
    
      bar_plot_df = df.groupby('y_true')[variable].value_counts().unstack(0)
      x_ticks = bar_plot_df.index.values
      control_values = bar_plot_df[0].values
      vfp_values = bar_plot_df[1].values
      control_values_all.append(control_values)
      vfp_values_all.append(vfp_values)

  control_values_avg = np.mean(control_values_all,axis=0)
  vfp_values_avg = np.mean(vfp_values_all,axis=0)

  width = 0.35
  

  rects1 = plt.bar(x_ticks - width/2, control_values_avg, width, label='Controls', alpha = 1, color = 'sandybrown')
  
  if min_and_max:
    lower_error =  control_values_avg - np.min(np.array(control_values_all),axis=0)
    upper_error =  np.max(np.array(control_values_all),axis=0) - control_values_avg 
    asymmetric_error = np.array(list(zip(lower_error, upper_error))).T
    plt.errorbar(x_ticks - width/2, control_values_avg, yerr=asymmetric_error,
                ecolor = 'gray',linestyle='',)
  else:
    plt.errorbar(x_ticks - width/2, control_values_avg, yerr=np.std(control_values_all,axis=0),
                ecolor = 'gray',linestyle='',)
  
  
  
  
  rects2 = plt.bar(x_ticks + width/2, vfp_values_avg, width, label='VFP', alpha = 1, color = 'lightskyblue')
  if min_and_max:
    lower_error =  vfp_values_avg - np.min(np.array(vfp_values_all),axis=0)
    upper_error =  np.max(np.array(vfp_values_all),axis=0) - vfp_values_avg 
    asymmetric_error = np.array(list(zip(lower_error, upper_error))).T
    plt.errorbar(x_ticks + width/2,vfp_values_avg, yerr=asymmetric_error,
                ecolor = 'gray',
                 linestyle='',
                 )
    
  else:
    plt.errorbar(x_ticks + width/2, vfp_values_avg, yerr=np.std(vfp_values_all,axis=0),  
                 linestyle='',
                 ecolor = 'gray')
  
  
  plt.ylim(0,80)

  if variable == 'noise':        
    plt.xticks(ticks = x_ticks, labels = ['None',"Some","High"])
    plt.xlabel('Background noise')

  if variable == 'vfp':        
    plt.xticks(ticks = x_ticks, labels = ['Controls', 'UVFP'])
    plt.xlabel('Diagnosis')

  if variable == 'loudness (recording)':        
    plt.xticks(ticks = x_ticks, labels = ['Low', 'Medium', 'High'])
    plt.xlabel('Recording loudness')

  if i == 1:
    plt.legend()
  if i in [1,2]:
    plt.yticks([], [])
  if i ==0:
    plt.ylabel('Count')

plt.tight_layout()
plt.savefig(output_dir+'/../annotations_figures/'+f'ordinal_aggregated.png', dpi=300)
plt.show()






In [ ]:
df1 = dfs.get('ps')
df2 = dfs.get('cjs')
df3 = dfs.get('ch')
print(df1.shape,df2.shape,df3.shape,)
df_all = df1.merge(df2, on=['file','y_true'], how='inner', suffixes=('_1', '_2')).merge(df3, on=['file','y_true'], how='inner', suffixes=('', '_3'))
df_all.columns = [n+'_3' if '_' not in n else n for n in df_all.columns]
df_all

In [ ]:
df_longform = []
for i in [1,2,3]:
  df_annotator_i = df_all[['y_true',f'noise_{i}',f'vfp_{i}', f'severity_{i}', f'roughness_{i}',	f'breathiness_{i}',	f'strain_{i}',	f'pitch_{i}', f'loudness (in person)_{i}', f'loudness (recording)_{i}']]
  df_annotator_i.columns = ['Diagnosis','Noise', 'UVFP Rating', 'Severity', 'Roughness', 'Breathiness', 'Strain', 'Pitch', 'Loudness', 'Loudness recording']

  df_annotator_i['Annotator'] = [f'Annotator {i}']*df_annotator_i.shape[0]
  df_longform.append(df_annotator_i)
df_longform = pd.concat(df_longform,axis=0).reset_index(drop = True)
df_longform



In [ ]:
df_small = df_longform[['Diagnosis','Severity', 'Roughness',
       'Breathiness', 'Strain', 'Pitch', 'Loudness',]]

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

for row in [0,1]:
  for col in [0,1,2]:
    variables = np.array([['Severity', 'Roughness', 'Breathiness'],
               ['Strain', 'Pitch', 'Loudness']])
               
    sns.kdeplot(ax=axes[row, col], data = df_small,x = variables[row, col], hue='Diagnosis', fill = True, legend= False)
    if row == 1 and col == 3:
      plt.legend()
    

plt.show()

In [ ]:
run_this = True
if run_this:
  sns.pairplot(df_small, hue="Diagnosis", 
              kind='scatter',
              diag_kind = "kde",
              corner=True,
              #  markers=["o", "s", "D"],
              plot_kws=dict(s=30, 
              #               #  edgecolor="white", 
                            #  linewidth=2.5, 
                            alpha=0.7)
              )


  # plt.savefig(output_dir+"pairplot_diagnosis.png", dpi=300)              
  plt.show()             

In [ ]:
from scipy import stats
def corrfunc(x, y, **kws):
    rho, _ = stats.pearsonr(x, y)
    ax = plt.gca()
    ax.annotate("r={:.2f}".format(rho),
                xy=(.1, .9), xycoords=ax.transAxes)


In [ ]:
df_small = df_longform[['Annotator','Diagnosis','Severity', 'Roughness',
       'Breathiness', 'Strain', 'Pitch', 'Loudness',]].dropna(axis=0)

In [ ]:
def corrdot(*args, **kwargs):
    corr_r = args[0].corr(args[1], 'spearman')
    corr_text = f"{corr_r:2.2f}".replace("0.", ".")
    ax = plt.gca()
    ax.set_axis_off()    
    marker_size = abs(corr_r) * 10000
    
    ax.scatter([.5], [.5], marker_size, [corr_r], alpha=0.6, cmap="coolwarm",
               vmin=-1, vmax=1, transform=ax.transAxes)
    font_size = 40 #abs(corr_r) * 40 + 5
    ax.annotate(corr_text, [.5, .5,],  xycoords="axes fraction",
                ha='center', va='center', fontsize=font_size)
    



In [ ]:
run_toy = True
# Pairwise correlation plot
sns.set(style='white', font_scale=1.6)
if run_toy:
  g = sns.PairGrid(df_small.drop('Diagnosis',axis=1).sample(frac=0.1), aspect=1.4, diag_sharey=False)
else:
  g = sns.PairGrid(df_longform.drop('Diagnosis',axis=1), aspect=1.4, diag_sharey=False)
g.map_lower(sns.regplot, lowess=True, ci=True, line_kws={'color': 'black'}, fit_reg=True,
          x_jitter=.1, y_jitter=.1, 
            scatter_kws={"s": 1, "alpha":0.1}
            )
g.map_diag(sns.distplot, kde_kws={'color': 'black'})
g.map_upper(corrdot)
plt.show() 

In [ ]:
sns.pairplot(df_longform, hue="Annotator", 
             kind='scatter',
             diag_kind = "kde",
             corner=True,
             markers=["o", "s", "D"],
             plot_kws=dict(s=30, 
            #               #  edgecolor="white", 
                          #  linewidth=2.5, 
                           alpha=0.7)
             )
             

plt.savefig(output_dir+"pairplot_annotator.png", dpi=300)              
plt.show() 

# Plots grouping annotators

In [ ]:
run_this = False

if run_this:
  plt.rcParams["figure.figsize"] = (12,8)
  plt.rcParams.update({'font.size': 18})


  for variable in df.columns:
    if variable in ['file', 'y_true']:
        continue
    for i, (annotator, df) in enumerate(dfs.items()):
      if annotator == 'ps':
        annotator = 'Annotator 1'
      elif annotator == 'cjs':
        annotator = 'Annotator 2'
      elif annotator == 'ch':
        annotator = 'Annotator 3'
      
      
      

      
      if variable in ['noise', 'vfp', 'loudness (recording)']:
        plt.subplot(1, 3, i+1)
        bar_plot_df = df.groupby('y_true')[variable].value_counts().unstack(0)
        x_ticks = bar_plot_df.index.values
        control_values = bar_plot_df[0].values
        vfp_values = bar_plot_df[1].values
        
        width = 0.35
        rects1 = plt.bar(x_ticks - width/2, control_values, width, label='Controls')
        rects2 = plt.bar(x_ticks + width/2, vfp_values, width, label='VFP')
        plt.title(f'{annotator}', size = 20)  
        if i+1==1:
          plt.ylabel('Count')
        if i+1==2:
          plt.xlabel(variable.capitalize())
        if i+1==3:
          plt.legend()
        plt.ylim(0,76)
          
        

      else:
        plt.subplot(1, 3, i+1)
        x0 = df[df['y_true']==0][variable].values
        x1 = df[df['y_true']==1][variable].values
        alpha = 0.7
        plt.hist(x0, bins=10, alpha=alpha, label = 'Controls')
        plt.hist(x1, bins=10, alpha=alpha, label = 'VFP')
        # plt.legend()
        plt.title(f'{annotator}', size = 20)  
        if i+1==1:
          plt.ylabel('Count')
        if i+1==2:
          plt.xlabel(variable.capitalize())
        if i+1==3:
          plt.legend()
        plt.ylim(0,76)


    plt.show()
    
    




In [ ]:
# Import pandas
import pandas as pd
# Import rpy2 for dataframe conversion
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from rpy2.robjects import globalenv

def python_to_r(df, df_name = 'df'):
  # Convert the python dataframe to the R dataframe
  with localconverter(ro.default_converter + pandas2ri.converter):
    dfr = ro.conversion.py2rpy(df)
  # Create a variable name in R's global environment
  globalenv[df_name] = dfr
  return

def r_to_python(df):
  # Convert R Dataframe to python dataframe
  with localconverter(ro.default_converter + pandas2ri.converter):
    dfpd = ro.conversion.rpy2py(df)
  return

In [ ]:
%%R
install.packages("irr")
library("irr")

In [ ]:

for variable in df1.columns:
  if variable in ['file', 'y_true']:
      continue
  df_var = df_all[[variable+'_1',variable+'_2',variable+'_3']]
  df_var = df_var.dropna(axis=0)
  python_to_r(df_var, 'df_var')
  print(variable, '='*30)
  if variable == 'vfp':    
    %R print(kappam.light(df_var))
    %R print(icc(df_var))
  else:
    %R print(icc(df_var))
  print()
  

In [ ]:
dropped_nans = {}
for variable in df.columns:
  if variable in ['file', 'y_true']:
      continue
  df_var = df_all[[variable+'_1',variable+'_2',variable+'_3']]
  rows_original = df_var.shape[0]
  df_var = df_var.dropna(axis=0)
  rows_after_removing_nan = df_var.shape[0]
  dropped_rows = rows_original-rows_after_removing_nan
  dropped_nans[variable] = dropped_rows
  python_to_r(df_var, 'df_var')
  print(variable, '='*30)
  if variable == 'vfp':    
    %R print(kappam.light(df_var))
  else:
    %R print(icc(df_var))
  print()
  

0 (0%) to 6 (4%) of samples were dropped because one of the three raters did not respond. 

# Performance metrics humans 



In [ ]:
plt.rcParams["figure.figsize"] = (4,4)
plt.rcParams.update({'font.size': 18})


results = {}

for i, (annotator, df) in enumerate(dfs.items()):
    if annotator == 'ps':
      annotator = 'Annotator 1'
    elif annotator == 'cjs':
      annotator = 'Annotator 2'
    elif annotator == 'ch':
      annotator = 'Annotator 3'

    print('===', annotator)

    y_pred = df.vfp.values
    print(np.unique(y_pred))
    y_pred[y_pred == 2] = 1
    y_true = [0 if 'Norm' in n else 1 for n in df.file.values ]
    df['y_true'] = y_true


    roc_auc = roc_auc_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    cm = pd.DataFrame(cm, index=['actual_UVFP-', 'actual_UVFP+'], columns=['annotated_UVFP-','annotated_UVFP+'  ])
    
    sns.heatmap(cm,annot=True)
    plt.xticks(rotation=45)
    plt.show()
    cr = classification_report(y_true, y_pred)


    sensitivity = recall_score(y_true, y_pred, pos_label=1)
    specificity = recall_score(y_true, y_pred, pos_label=0)

    print(cm)
    print(cr)
    metrics = [ roc_auc, sensitivity, specificity]
    names = [ 'roc_auc', 'sensitivity', 'specificity']
    for m, n in zip(metrics, names):
      print(n, np.round(m,2))

    results[annotator]=metrics


In [ ]:
results = pd.DataFrame(results, index = names).round(2)
results['Avg.'] = results.mean(axis=1)
results.T

In [ ]:
print('controls having loudness (2 or 3 out of 3):', np.round(49/77*100,1),'%')
print('UVFP having loudness (2 or 3 out of 3):', np.round(60/74*100,1),'%')

print('controls having hi loudness (3 out of 3):', np.round(6/77*100, 1), '%')
print('UVFP having hi loudness (3 out of 3):', np.round(44/74*100,1),'%')

In [ ]:

print('controls having inferred loudness in person (2 or 3 out of 3):', np.round(49/77*100,1),'%')
print('UVFP having loudness (2 or 3 out of 3):', np.round(60/74*100,1),'%')

print('controls having hi loudness (3 out of 3):', np.round(6/77*100, 1), '%')
print('UVFP having hi loudness (3 out of 3):', np.round(44/74*100,1),'%')